# (Quickly) Export to CATMAID
---

In [1]:
import os
import sys
import re
import subprocess
from pathlib import Path
from functools import partial
from shutil import rmtree

from ruamel.yaml import YAML
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from skimage.io import imsave

import renderapi
from renderapi.client import ArgumentParameters

In [2]:
owner = 'rlane'
project = '20190517_UMCG_RL005'

section_thickness = 80  # nm

In [3]:
project_dir = Path(f'/long_term_storage/{owner}/SECOM/projects/{project}/')
export_dir = Path(f'/long_term_storage/{owner}/CATMAID/projects/{project}/')

# Render parameters
render_connect_params = {
    'host': 'sonic',
    'port': 8080,
    'owner': owner,
    'project': project,
    'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
    'memGB': '2G'
}
# Create a renderapi.connect.Render object
render = renderapi.connect(**render_connect_params)
# Collect stacks
stacks_all = renderapi.render.get_stacks_by_owner_project(render=render)
stacks_EM = [stack for stack in stacks_all if 'EM' in stack]
stacks_FM = [stack for stack in stacks_all if stack not in stacks_EM]
stacks_2_export = [stack for stack in stacks_all if ('montaged' in stack) or \
                                                    ('overlaid_correlated' in stack)]

out = f"""\
owner............... {owner}
project............. {project}
project directory... {project_dir}
export directory.... {export_dir}
stacks.............. {stacks_all}
EM stacks........... {stacks_EM}
FM stacks........... {stacks_FM}
Export stacks....... {stacks_2_export}
"""
print(out)

owner............... rlane
project............. 20190517_UMCG_RL005
project directory... /long_term_storage/rlane/SECOM/projects/20190517_UMCG_RL005
export directory.... /long_term_storage/rlane/CATMAID/projects/20190517_UMCG_RL005
stacks.............. ['hoechst_overlaid_correlated', 'big_EM_correlated', 'mm_EM', 'hoechst_overlaid', 'lil_EM_montaged', 'lil_EM', 'hoechst', 'big_EM']
EM stacks........... ['big_EM_correlated', 'mm_EM', 'lil_EM_montaged', 'lil_EM', 'big_EM']
FM stacks........... ['hoechst_overlaid_correlated', 'hoechst_overlaid', 'hoechst']
Export stacks....... ['hoechst_overlaid_correlated', 'lil_EM_montaged']



In [4]:
stack_data = pd.DataFrame(columns=['stack', 'section', 'z'])

i = 0
for stack in stacks_2_export:

    # Get z values for each stack
    z_vals = renderapi.stack.get_z_values_for_stack(stack, render=render)
    # Loop through z values
    for z in z_vals:

        # Get sectionId for z value
        section = renderapi.stack.get_sectionId_for_z(stack, z, render=render)

        # Build up stack data
        stack_data.loc[i, 'stack'] = stack
        stack_data.loc[i, 'z'] = z
        stack_data.loc[i, 'section'] = section

        # Increment index
        i += 1

stack_data.infer_objects()

,stack,section,z
0,hoechst_overlaid_correlated,S001,1.0
1,lil_EM_montaged,S001,1.0


#### Aggregate stack stack data

In [5]:
stack_data_agg = stack_data.groupby('stack').agg(list)
stack_data_agg

,section,z
stack,,
hoechst_overlaid_correlated,[S001],[1.0]
lil_EM_montaged,[S001],[1.0]


## Export to CATMAID
---
### Set up CATMAID export parameters

In [6]:
class CatmaidBoxesParameters(ArgumentParameters):
    """
    """
    def __init__(self, stack, root_directory,
                 height=1024, width=1024, fmt='png', max_level=0,
                 host=None, port=None, baseurl=None,
                 owner=None, project=None, render=None, **kwargs):
        
        super(CatmaidBoxesParameters, self).__init__(**kwargs)
        
        self.stack = stack
        self.rootDirectory = root_directory
        self.height = height
        self.width = width
        self.format = fmt
        self.maxLevel = max_level
        
        render_kwargs = render.make_kwargs()
        host = render_kwargs.get('host')
        port = render_kwargs.get('port')
        self.baseDataUrl = renderapi.render.format_baseurl(host, port)
        self.owner = render_kwargs.get('owner') if owner is None else owner
        self.project = render_kwargs.get('project') if project is None else project

Ideally `max_level` is set such that

\begin{equation}
\left( \frac{w_s}{w_t \,\, 2^m} \right) < 1
\end{equation}

where $m$ is `max_level`, $w_s$ is the width of the stack and $w_t$ is the width of each tile. Then

\begin{equation}
m = \textrm{ceil} \left( \log{\frac{w_s}{w_t}} \times \frac{1}{\log{2}} \right)
\end{equation}

In [8]:
stack_data_agg['export_params'] = None

i = 0
for stack in stack_data_agg.index:
    
    # Determine `max_level` such that the full section is in view when you fully zoom out
    width_tile = 1024
    height_tile = 1024
    stack_bounds = renderapi.stack.get_stack_bounds(stack, render=render)
    width_stack = max(stack_bounds['maxX'] - stack_bounds['minX'],
                      stack_bounds['maxY'] - stack_bounds['minY'])
    max_level = int(np.ceil(np.log(width_stack/width_tile) * 1/np.log(2)))
    
    # Set parameters for CATMAID export
    export_params = CatmaidBoxesParameters(stack=f'{stack}',
                                           root_directory=export_dir.parent.as_posix(),
                                           width=width_tile,
                                           height=height_tile,
                                           max_level=max_level,
                                           project=f'{project}',
                                           render=render)
    # Add to stack DataFrame
    stack_data_agg.loc[stack, 'export_params'] = export_params
    
# Preview export parameters
list(stack_data_agg.sample(1)['export_params'].values[0].to_java_args())

['--stack',
 'hoechst_overlaid_correlated',
 '--rootDirectory',
 '/long_term_storage/rlane/CATMAID/projects',
 '--height',
 '1024',
 '--width',
 '1024',
 '--format',
 'png',
 '--maxLevel',
 '8',
 '--baseDataUrl',
 'http://sonic:8080/render-ws/v1',
 '--owner',
 'rlane',
 '--project',
 '20190517_UMCG_RL005']

### Call render script
`render_catmaid_boxes.sh`
```sh
Usage: java -cp <render-module>-standalone.jar
      org.janelia.render.client.BoxClient [options] Z values for layers to
      render
  Options:
  * --baseDataUrl
      Base web service URL for data (e.g. http://host[:port]/render-ws/v1)
    --binaryMask
      use binary mask (e.g. for DMG data)
      Default: false
    --createIGrid
      create an IGrid file
      Default: false
    --doFilter
      Use ad hoc filter to support alignment
      Default: false
    --filterListName
      Apply this filter list to all rendering (overrides doFilter option)
    --forceGeneration
      Regenerate boxes even if they already exist
      Default: false
    --format
      Format for rendered boxes
      Default: png
  * --height
      Height of each box
    --help
      Display this note
    --label
      Generate single color tile labels instead of actual tile images
      Default: false
    --maxLevel
      Maximum mipmap level to generate
      Default: 0
    --maxOverviewWidthAndHeight
      Max width and height of layer overview image (omit or set to zero to
      disable overview generation)
    --numberOfRenderGroups
      Total number of parallel jobs being used to render this layer (omit if
      only one job is being used)
  * --owner
      Stack owner
  * --project
      Stack project
    --renderGroup
      Index (1-n) that identifies portion of layer to render (omit if only one
      job is being used)
  * --rootDirectory
      Root directory for rendered tiles (e.g.
      /tier2/flyTEM/nobackup/rendered_boxes)
    --skipInterpolation
      skip interpolation (e.g. for DMG data)
      Default: false
  * --stack
      Stack name
  * --width
      Width of each box
```

In [9]:
def run_render_catmaid_boxes(z, path_to_script, java_args):
    """Wrapper for `render_catmaid_boxes` script to enable multiprocessing"""
    p = subprocess.run([path_to_script.as_posix(), f'{z:.0f}'] + java_args)

In [10]:
# Path to `render_catmaid_boxes` shell script
sh = Path(render_connect_params['client_scripts']) / 'render_catmaid_boxes.sh'
# Set number of cores for multiprocessing
N_cores = 5

for stack in tqdm_notebook(stack_data_agg.index):

    # Set export parameters
    export_params = stack_data_agg.loc[stack, 'export_params']
    java_args = list(export_params.to_java_args())

    # Get z values for particular stack
    z_values = renderapi.stack.get_z_values_for_stack(stack, 
                                                      render=render)

    # Set up `render_catmaid_boxes` client
    render_catmaid_boxes_partial = partial(run_render_catmaid_boxes,
                                           path_to_script=sh,
                                           java_args=java_args)

    # Run `render_catmaid_boxes` across `N_cores`
    with renderapi.client.WithPool(N_cores) as pool:
        pool.map(render_catmaid_boxes_partial, z_values)

### Resort CATMAID tiles
By (unchangeable) default, `render_catmaid_boxes` exports tiles as

    `root directory` / `project` / `stack` / `width`x`height` / `zoom level` / `z` / `row` / `col`.`ext`

This is ok, but preferred format for importing to CATMAID is [tile source convention 1](https://catmaid.readthedocs.io/en/stable/tile_sources.html#tile-source-types) --- "[File-based image stack](https://catmaid.readthedocs.io/en/stable/tile_sources.html#file-based-image-stack)"

    `root directory` / `project` / `stack` / `z` / `row`_`col`_`zoom level`.`ext`

In [11]:
stack_dirs = [f for f in export_dir.glob('*/') if f.is_dir()]
for stack_dir in tqdm_notebook(stack_dirs):
    
    stack = stack_dir.name
    tile_fns = stack_dir.glob('1024x1024/**/[0-9]*.png')
    
    for tile_fn in tile_fns:
        
        # Extract info from filename
        zoom_level = int(tile_fn.parent.parent.parent.name)
        z = int(tile_fn.parent.parent.name)
        row = int(tile_fn.parent.name)
        col = int(tile_fn.stem)
        ext = tile_fn.suffix
        
        # Reformat tile
        tile_format_1 = stack_dir / f'{z}/{row}_{col}_{zoom_level}{ext}'
        
        # Rename tile with new format
        tile_format_1.parent.mkdir(parents=True, exist_ok=True)
        tile_fn.rename(tile_format_1)
        
    # Clean up (now empty) directory tree
    rmtree((stack_dir / '1024x1024').as_posix())

### Make thumbnails

In [12]:
stack_dirs = [fp for fp in export_dir.glob('*/') if fp.is_dir()]
for stack_dir in tqdm_notebook(stack_dirs):

    # Get stack and z value
    stack = stack_dir.name
    z = int(next(stack_dir.iterdir()).name)
    # Set proper scale for section image
    bounds = renderapi.stack.get_bounds_from_z(stack=stack, z=z, render=render)
    width = bounds['maxX'] - bounds['minX']
    scale = 128 / width
    # Render section image
    thumb = renderapi.image.get_section_image(stack=stack,
                                              z=z,
                                              scale=scale,
                                              render=render)

    # Copy thumbnail to each section directory
    for section_dir in stack_dir.glob('*/'):
        fp = section_dir / 'small.png'
        imsave(fp, thumb)

## Create project yaml file
---

In [13]:
# Set project yaml file
project_yaml = export_dir / 'project.yaml'

stack_data = []
for stack_dir in stack_dirs:
    stack = stack_dir.name

    # Get dimension data
    bounds = renderapi.stack.get_stack_bounds(stack, render=render)
    dimension = (int(bounds['maxX'] - bounds['minX'] + 1),
                 int(bounds['maxY'] - bounds['minY'] + 1),
                 int(bounds['maxZ'] - bounds['minZ'] + 1))

    # Get resolution data
    res_data = renderapi.stack.get_full_stack_metadata(stack, render=render)['currentVersion']
    resolution = (np.round(res_data['stackResolutionX'], 3),
                  np.round(res_data['stackResolutionY'], 3),
                  np.round(res_data['stackResolutionZ'], 3))
    
    # Project data for output to project yaml file
    stack_datum = {
        "title": f"{stack}",
        "dimension": f"{dimension}",
        "resolution": f"{resolution}",
        "zoomlevels": 7,
        "mirrors": [{
            "title": "Great title",
            "tile_width": 1024,
            "tile_height": 1024,
            "tile_source_type": 1,
            "fileextension": "png",
            "url": f"http://sonic{(export_dir/stack).as_posix()}"
        }]
    }
    
    stack_data.append(stack_datum)

# Aggregate stack data into project for output to project yaml file
project_data = {
    "project": {
        "title": f"{project}",
        "stacks": stack_data
    }
}

In [14]:
yaml = YAML()
yaml.indent(mapping=2, offset=0)
yaml.dump(project_data, project_yaml)
yaml.dump(project_data, sys.stdout)

project:
  title: 20190517_UMCG_RL005
  stacks:
  - title: hoechst_overlaid_correlated
    dimension: (171078, 172300, 1)
    resolution: (1.0, 1.0, 1.0)
    zoomlevels: 7
    mirrors:
    - title: Great title
      tile_width: 1024
      tile_height: 1024
      tile_source_type: 1
      fileextension: png
      url: http://sonic/long_term_storage/rlane/CATMAID/projects/20190517_UMCG_RL005/hoechst_overlaid_correlated
  - title: lil_EM_montaged
    dimension: (134316, 123126, 1)
    resolution: (1.0, 1.0, 1.0)
    zoomlevels: 7
    mirrors:
    - title: Great title
      tile_width: 1024
      tile_height: 1024
      tile_source_type: 1
      fileextension: png
      url: http://sonic/long_term_storage/rlane/CATMAID/projects/20190517_UMCG_RL005/lil_EM_montaged
